In [1]:
import pandas as pd

- Import data.

In [2]:
change_frame = pd.read_csv('data/raw_changemyview_initial_scrape')
opinion_frame = pd.read_csv('data/raw_unpopularopinion_initial_scrape')

C:\Users\Avanyali\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3166: DtypeWarning: Columns (1,2,5,7,9,10,11,12,13,17,19,20,21,22,23,25,26,27,28,29,30,31,32,36,38,43,44,49,50,54,57,60,61,66,69,71,74,75,80,81,83,85,86,89,92,93,95,96,99,100) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Avanyali\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3166: DtypeWarning: Columns (1,2,5,7,9,10,11,12,17,19,20,21,22,23,25,26,27,28,29,30,31,32,36,38,42,46,47,52,56,59,61,62,63,64,66,67,68,70,72,73,76,77,78) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


***Data Cleaning***

- Combine frames for cleaning.

In [3]:
df = pd.concat([change_frame, opinion_frame])

- Filter out fully null columns and posts that are likely made with moderation in mind.

In [4]:
all_nas = df.isna().sum() == df.shape[0]
df.drop(columns = [key for key, value in all_nas.to_dict().items() if value], inplace = True)

In [5]:
df = df[df['pinned'] != True]
df = df[df['stickied'] != True]

- Reduce dataframe to a selection of columns.

In [6]:
relevant_columns = ['author', 'author_fullname', 'id', 'is_self', 'num_comments', 'selftext', 'score', 'subreddit', 'title']

In [7]:
df = df[relevant_columns]

- Remove posts that are not self-posted text posts.

In [8]:
df = df[df['is_self'] == True]

In [9]:
df.drop(columns = 'is_self', inplace = True)

- Remove posts without body text. Confirm it does not imbalance our classes.

In [10]:
df = df[df['selftext'].isna().map(lambda x: False if x == True else True)]

In [11]:
df['subreddit'].value_counts(normalize = True)

unpopularopinion    0.503841
changemyview        0.496159
Name: subreddit, dtype: float64

- Check nulls.

In [12]:
df.isna().sum()

author                0
author_fullname    6443
id                    0
num_comments          0
selftext              0
score                 0
subreddit             0
title                 0
dtype: int64

In [13]:
df[df['author'] == '[deleted]']['author_fullname'].isna().sum()

3445

In [14]:
df[df['author'] == '[deleted]']['author_fullname'].shape

(3445,)

- Author fullname was hoped to be usable when author was '[deleted]'. It is not, and will be dropped.

In [15]:
df.drop(columns = 'author_fullname', inplace = True)

- Drop all posts with '[deleted]' or '[removed]' as text data, check if class imbalance is created.

In [16]:
df = df[df['selftext'] != '[deleted]']

In [17]:
df = df[df['selftext'] != '[removed]']

In [18]:
df['subreddit'].value_counts(normalize = True)

unpopularopinion    0.521837
changemyview        0.478163
Name: subreddit, dtype: float64

- Remove automoderator posts and moderator posts.
- Moderator names from:
   - https://www.reddit.com/r/unpopularopinion/about/moderators
   - https://www.reddit.com/r/changemyview/about/moderators

In [19]:
df = df[df['author'] != 'AutoModerator']

In [20]:
unpop_mods = ['Young_Zaphod','TheUniquestUsername','Kinrove','conalfisher','RandomName01','dantheman280','AutoModerator','UnpopularOpinionMods','MisfitPotatoReborn','Umbresp','Blank-Cheque','siouxsie_siouxv2','Iangator','sibre2001','justcool393','ignorantConservative','BotTerminator','SendWhiskey','anticapitalist','Flair_Helper','Wismuth_Salix','Blubatt','theimperious1','BlatantConservative','nexerxe','dramasutra2020','Cambino_02']

In [21]:
cmv_mods = ['Ansuz07','cwenham','Nepene','hacksoncode','huadpe','FlyingFoxOfTheYard_','thedylanackerman','Huntingmoa','Jaysank','tbdabbholm','Armadeo','Poo-et','themaskedserpent','Helpfulcloning','Guanfranco','ViewedFromTheOutside','ZeroPointZero_','entpmisanthrope','TheRegen','ihatedogs2','DeleteriousEuphuism','SquibblesMcGoo','Znyper','hallidev','qtx','DeltaBot','AutoModerator','herrsatan','LetMeNotHear','redditor427']

In [22]:
df = df[df['author'].map(lambda x: True if x not in unpop_mods else False)]

In [23]:
df = df[df['author'].map(lambda x: True if x not in cmv_mods else False)]

In [24]:
df['subreddit'].value_counts(normalize = True)

unpopularopinion    0.51815
changemyview        0.48185
Name: subreddit, dtype: float64

- Drop duplicate rows.

In [25]:
df.drop_duplicates(inplace = True)

In [26]:
df['subreddit'].value_counts(normalize = True)

unpopularopinion    0.563229
changemyview        0.436771
Name: subreddit, dtype: float64

- Save cleaned file.

In [27]:
df.to_csv('data/cleaned_cmv&unpop_data')